# $\ell^2$-Regularized CNNs Learn Gabors on MNIST

In [ ]:
%env CUDA_VISIBLE_DEVICES=1

import tensorflow as tf

import numpy as np

import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

## Helper Functions

In [ ]:
# define some helper functions to abstract away the tensorflow interface
## and make the network-building code cleaner

def weight_variable(shape,name):
    initial = tf.truncated_normal(shape, stddev=0.1, name=name)
    return tf.Variable(initial)

def bias_variable(shape,name):
    initial = tf.constant(0.1, shape=shape, name=name)
    return tf.Variable(initial)

def conv2d(x, W,name):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME', name=name)

def max_pool_2x2(x,name):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME', name=name)

In [ ]:
def specifyGraph(weight_decay=2e-1):
    
    graph = tf.Graph()
    
    with graph.as_default():
        
        x = tf.placeholder(tf.float32, shape=[None, 784],name="inputs")
        y_ = tf.placeholder(tf.float32, shape=[None, 10],name="targets")

        x_image = tf.reshape(x, [-1, 28, 28, 1],name="reshaped_input")

        keep_prob = tf.placeholder(tf.float32,name="dropout_keep_probability")

        ### Convolution 1
        W_conv1 = weight_variable([9, 9, 1, 32],'weights_convolution_1')
        b_conv1 = bias_variable([32],'biases_convolution_1')

        h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1,'output_convolution_1') 
                                     + b_conv1,
                            name="activations_convolution_1")

        h_pool1 = max_pool_2x2(h_conv1,"activations_pool_1")

        ### Convolution 2

        W_conv2 = weight_variable([5, 5, 32, 64],'weights_convolution_2')
        b_conv2 = bias_variable([64], 'biases_convolution_2')

        h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2,'output_convolution_2') 
                                     + b_conv2,
                            name="activations_convolution_2")

        h_pool2 = max_pool_2x2(h_conv2,"activations_pool_2")

        ### Fully-Connected 1

        W_fc1 = weight_variable([7 * 7 * 64, 1024],'weights_fullyconnected_1')
        b_fc1 = bias_variable([1024],'biases_fullyconnected_1')

        h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64],
                                 name="reshapedactivations_hidden_2")
        h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1,
                                    name="output_fullyconnected_1") 
                           + b_fc1,
                          name="activations_fullyconnected_1")


        h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob,
                                  name="droppedactivations_fullyconnected_1")

        ### Fully-Connected 2

        W_fc2 = weight_variable([1024, 10],"weights_fullyconnected_2")
        b_fc2 = bias_variable([10], "biases_fullyconnected_2")

        y_conv = tf.add(tf.matmul(h_fc1_drop, W_fc2,
                          name="outputs_fullyconnected_2"),
                        b_fc2,name="activations_fullyconnected_2")

        cross_entropy = tf.reduce_mean(
                            tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv,
                                                                       name="crossentropy_cost"),
                                name="mean_crossentropy_cost")

        weight_decay = tf.reduce_sum([tf.nn.l2_loss(weight_decay*weight) # .2, 10k; .1, 25k
                              for weight in [W_conv1,W_conv2,W_fc1,W_fc2]])

        total_cost = cross_entropy + weight_decay

        train_step = tf.train.AdamOptimizer(1e-4).minimize(total_cost)

        correct_prediction = tf.equal(tf.argmax(y_conv, 1,
                                               name="MAPestimate_index"), 
                                      tf.argmax(y_, 1,
                                               name="label_index"),
                                     name="MAP_iscorrect")

        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32),
                             name="fraction_correct")

        operations = {'inputs':x,
                      'targets':y_,
                      'keep_prob':keep_prob,
                      'accuracy':accuracy,
                      'output':y_conv,
                      'train_step':train_step,
                      'filters':W_conv1
                      }
    
    return graph, operations

In [ ]:
def runTrain(graph,num_minibatches,batch_size=64):
    
    with tf.Session(graph=graph) as sess:
        sess.run(tf.global_variables_initializer())
        x = ops['inputs']; y_ = ops['targets']
        accuracy = ops['accuracy']
        train_step = ops['train_step']
        keep_prob = ops['keep_prob']
        filters = ops['filters']

        for i in range(num_minibatches):
            batch = mnist.train.next_batch(batch_size)
            if i % 500 == 0:
                train_accuracy = accuracy.eval(feed_dict={
                                    x: batch[0], y_: batch[1], keep_prob: 1.0})
                print('step %d, training accuracy %g' % (i, train_accuracy))

            train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

        print('test accuracy %g' % accuracy.eval(feed_dict={
                    x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))
        
        weights = filters.eval()
    
    return weights

In [ ]:
regularized_graph, ops = specifyGraph(weight_decay=2e-1)

In [ ]:
regularized_weights = runTrain(graph,10000)

In [ ]:
unregularized_graph, ops = specifyGraph(weight_decay=0.)

unregularized_weights = runTrain(unregularized_graph,10000)

In [ ]:
def plotFilters(filterBank,title=""):
    numFilters = filterBank.shape[-1]
    plt.figure(figsize=(6,6))

    for idx in range(numFilters):
        ax = plt.subplot(5,7,idx+1)
        filtr = filterBank[:,:,0,idx]
        ax.imshow(filtr,cmap='Greys',interpolation='None')
        ax.set_xticks([]); ax.set_yticks([]);
    plt.suptitle(title+" Filters",
                fontsize='xx-large',fontweight='bold');

In [ ]:
plotFilters(unregularized_weights, title="Unregularized")

In [ ]:
plotFilters(regularized_weights,title=r"$\ell^2$ Regularized")

In [ ]:
import scipy.signal

In [ ]:
gaussian_random_weights = np.random.standard_normal(size=regularized_weights.shape)

In [ ]:
def plotAutoCorrs(weights,names):
    
    plt.figure(figsize=(16,4));
    plt.subplot(1,len(weights),1)
    
    for idx,(weight,name) in enumerate(zip(weights,names)):
        ax = plt.subplot(1,len(weights),idx+1)
        plotAutoCorr(ax,weight,name)
    
    plt.tight_layout()
    plt.suptitle("Weight Auto-Correlations",
                fontsize='xx-large',fontweight='bold',
                 y=1.05,
                )
    return

def plotAutoCorr(ax,weight,name):
    
    auto_corr = scipy.signal.correlate2d(weight,weight,mode='same')
    auto_corr_shape = auto_corr.shape
    auto_corr = auto_corr.ravel()
    auto_corr[np.argmax(auto_corr)] = 0
    auto_corr = auto_corr.reshape(auto_corr_shape)
    
    ax.imshow(np.triu(auto_corr));
    ax.set_xticks([]); ax.set_yticks([]);
    plt.title(name,
             fontsize='x-large',
             fontweight='bold')
    return

In [ ]:
randomWeightIdx = np.random.choice(numFilters)
no_reg_weight = unregularized_weights[:,:,0,randomWeightIdx]
reg_weight = regularized_weights[:,:,0,randomWeightIdx]
randomized_weight = gaussian_random_weights[:,:,0,randomWeightIdx]

weights_to_show = [no_reg_weight, reg_weight, randomized_weight]
names = ["No Regularization", r"$\ell^2$ Regularization", "Gaussian Random"]

plotAutoCorrs(weights_to_show,names)